# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298446095009                   -0.85    4.1
  2   -8.300212704783       -2.75       -1.25    1.0
  3   -8.300450232113       -3.62       -1.89    4.1
  4   -8.300462251763       -4.92       -2.76    2.4
  5   -8.300464066870       -5.74       -3.05   11.5
  6   -8.300464349964       -6.55       -3.24    1.0
  7   -8.300464489357       -6.86       -3.38    3.2
  8   -8.300464566438       -7.11       -3.50    1.0
  9   -8.300464621837       -7.26       -3.69    1.0
 10   -8.300464638860       -7.77       -3.88    1.0
 11   -8.300464643826       -8.30       -4.36    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67490971302                   -0.70    7.0
  2   -16.67878093465       -2.41       -1.13    1.4
  3   -16.67923734593       -3.34       -1.85    3.1
  4   -16.67928086753       -4.36       -2.76    2.8
  5   -16.67928596577       -5.29       -3.22    5.1
  6   -16.67928621540       -6.60       -3.52    3.1
  7   -16.67928622251       -8.15       -3.96    1.0
  8   -16.67928622420       -8.77       -4.65    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32535913677                   -0.56    6.8
  2   -33.33268300820       -2.14       -1.00    1.0
  3   -33.33589115217       -2.49       -1.69    3.9
  4   -33.33688959099       -3.00       -2.47    3.2
  5   -33.33693454703       -4.35       -2.99    6.9
  6   -33.33694385929       -5.03       -3.80    5.8
  7   -33.33694392054       -7.21       -4.27    9.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298628665269                   -0.85    4.0
  2   -8.300293172362       -2.78       -1.59    1.0
  3   -8.300459932177       -3.78       -2.66    5.9
  4   -8.300430354595   +   -4.53       -2.49    9.0
  5   -8.300464539061       -4.47       -3.76    2.8
  6   -8.300464603704       -7.19       -4.01    5.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32527852374                   -0.56    6.8
  2   -33.25070635228   +   -1.13       -1.23    1.0
  3   +30.56347230336   +    1.80       -0.16   11.0
  4   -33.30756104328        1.81       -1.67    8.5
  5   -30.39125152299   +    0.46       -0.83    4.6
  6   -33.02860957009        0.42       -1.26    8.9
  7   -33.31960578953       -0.54       -1.81    4.5
  8   -33.33386549338       -1.85       -2.14    2.2
  9   -33.33421003409       -3.46       -2.03    3.1
 10   -33.33634312320       -2.67       -2.63    2.6
 11   -33.33683310461       -3.31       -2.94    2.8
 12   -33.33692302715       -4.05       -3.24    3.2
 13   -33.33694270810       -4.71       -3.52    3.9
 14   -33.33694321512       -6.29       -3.95    3.9
 15   -33.33693851093   +   -5.33       -3.69    4.0
 16   -33.33694290886       -5.36       -4.05    4.2


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.